# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Conv2D, Convolution2D, Permute, BatchNormalization, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [6]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [7]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
print(K.image_data_format())
if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

# Convolutional layers
model.add(Conv2D(32, (8, 8), padding="same", strides=(4, 4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding="same", strides=(2, 2)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", strides=(1, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(nb_actions))
model.add(Activation('linear'))

model.summary()

channels_last
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        8224      
_________________________________________________________________
activation_5 (Activation)    (None, 21, 21, 32)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 21, 21, 32)        128       
_________________________________________________________________
dropout_4 (Dropout)          (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        32832     
_________________________________________________________________
activation_6 (Activation)    (None, 11, 

2. Implementación de la solución DQN

In [8]:
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

In [9]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1., value_min=.1, value_test=.05, nb_steps=100)

In [10]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=50000, gamma=.99,
               target_model_update=10000,
               train_interval=1)
dqn.compile(Adam(learning_rate=.0001), metrics=['mae'])

In [11]:
weights_filename = 'dqn_SpaceInvaders-v0_weights_1.h5f'

In [12]:
# Training part
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=20000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(env, callbacks=callbacks, nb_steps=200000, log_interval=10000, visualize=False,verbose=2)
#dqn.load_weights(weights_filename)
#dqn.fit(env, callbacks=callbacks, nb_steps=240000, log_interval=10000, visualize=False, verbose=2)

dqn.save_weights(weights_filename, overwrite=True)

Training for 480000 steps ...


c:\Users\Usuario\anaconda3\envs\ar\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    420/480000: episode: 1, duration: 10.599s, episode steps: 420, steps per second:  40, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.479 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1131/480000: episode: 2, duration: 2.788s, episode steps: 711, steps per second: 255, episode reward:  7.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.421 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1941/480000: episode: 3, duration: 2.487s, episode steps: 810, steps per second: 326, episode reward: 10.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.410 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2809/480000: episode: 4, duration: 2.786s, episode steps: 868, steps per second: 312, episode reward: 11.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.386 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   3376/480000: episode: 5, duration: 1.910s, episode steps: 567, steps per

c:\Users\Usuario\anaconda3\envs\ar\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  50469/480000: episode: 73, duration: 14.276s, episode steps: 490, steps per second:  34, episode reward:  3.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 2.614 [0.000, 5.000],  loss: 1.989203, mae: 1.581231, mean_q: 2.504246, mean_eps: 0.849292
  51050/480000: episode: 74, duration: 15.113s, episode steps: 581, steps per second:  38, episode reward: 10.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.664 [0.000, 5.000],  loss: 1.858803, mae: 1.536352, mean_q: 2.417737, mean_eps: 0.847720
  51814/480000: episode: 75, duration: 20.104s, episode steps: 764, steps per second:  38, episode reward: 15.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.796 [0.000, 5.000],  loss: 1.843763, mae: 1.535755, mean_q: 2.422410, mean_eps: 0.845704
  52730/480000: episode: 76, duration: 23.760s, episode steps: 916, steps per second:  39, episode reward: 14.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.624 [0.000, 5.000],  loss: 1.834010, mae: 1.519053, mean_q: 2

In [16]:
# Testing part to calculate the mean reward

dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=20, visualize=False)

Testing for 20 episodes ...
Episode 1: reward: 5.000, steps: 380
Episode 2: reward: 23.000, steps: 1176
Episode 3: reward: 16.000, steps: 932
Episode 4: reward: 10.000, steps: 754
Episode 5: reward: 22.000, steps: 1251
Episode 6: reward: 6.000, steps: 649
Episode 7: reward: 18.000, steps: 1011
Episode 8: reward: 7.000, steps: 604
Episode 9: reward: 13.000, steps: 887
Episode 10: reward: 12.000, steps: 711
Episode 11: reward: 10.000, steps: 810
Episode 12: reward: 16.000, steps: 957
Episode 13: reward: 15.000, steps: 936
Episode 14: reward: 16.000, steps: 943
Episode 15: reward: 7.000, steps: 418
Episode 16: reward: 6.000, steps: 342
Episode 17: reward: 11.000, steps: 800
Episode 18: reward: 22.000, steps: 1180
Episode 19: reward: 12.000, steps: 812
Episode 20: reward: 21.000, steps: 1130


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---